In [23]:
import sqlite3

# Connect to database (or create if it doesn't exist)
conn = sqlite3.connect("Data/aenc.sqlite")  # Replace with your database file

# Create a cursor object
cursor = conn.cursor()

In [24]:
cursor.execute("SELECT * FROM Bonus")
Bonus = cursor.fetchall()

cursor.execute("SELECT * FROM Customer")
Customer = cursor.fetchall()

cursor.execute("SELECT * FROM Department")
Department = cursor.fetchall()

cursor.execute("SELECT * FROM Employee")
Employee = cursor.fetchall()

cursor.execute("SELECT * FROM Product")
Product = cursor.fetchall()

cursor.execute("SELECT * FROM Sales_Order")
Sales_Order = cursor.fetchall()

cursor.execute("SELECT * FROM Sales_Order_Item")
Sales_Order_Item = cursor.fetchall()

cursor.execute("SELECT * FROM State")
State = cursor.fetchall()

In [30]:
import sqlite3
import psycopg2

# SQLite connection
sqlite_conn = sqlite3.connect("Data/aenc.sqlite")
sqlite_cursor = sqlite_conn.cursor()

# PostgreSQL connection
pg_conn = psycopg2.connect(
    host="132.164.25.250",
    port="5432",
    database="AenC",
    user="admin",
    password="MelleWessel123"
)
pg_cursor = pg_conn.cursor()

# Function to push data from SQLite to PostgreSQL
def push_data_to_pg(sqlite_cursor, pg_cursor, table_name):
    # Fetch data from SQLite table
    sqlite_cursor.execute(f"SELECT * FROM {table_name}")
    data = sqlite_cursor.fetchall()

    # If the table has data, proceed to insert into PostgreSQL
    if data:
        # Get column names of the table from SQLite
        sqlite_cursor.execute(f"PRAGMA table_info({table_name})")
        columns = [column[1] for column in sqlite_cursor.fetchall()]
        
        # Quote the table and column names to handle case sensitivity
        quoted_columns = [f'"{column}"' for column in columns]
        quoted_table_name = f'"{table_name}"'
        
        # Construct the INSERT SQL query
        insert_query = f"INSERT INTO {quoted_table_name} ({', '.join(quoted_columns)}) VALUES ({', '.join(['%s'] * len(columns))})"
        
        # Insert data row by row into PostgreSQL
        pg_cursor.executemany(insert_query, data)
        pg_conn.commit()

# List of tables to transfer
tables = ["Product", "Department", "State", "Customer", "Employee", "Bonus", "Sales_Order", "Sales_Order_Item"]

# Push data from each table
for table in tables:
    push_data_to_pg(sqlite_cursor, pg_cursor, table)

# Close connections
sqlite_conn.close()
pg_conn.close()


NumericValueOutOfRange: integer out of range
